# Activity proxies in Myanmar
## Introduction
This document present an analysis of Veraset data to be used as an estimation of Mobility in Myanmar

In [1]:
import pandas as pd
import dask.dataframe as dd
from datetime import timedelta
import plotly.express as px
import pyarrow as pa

In [2]:
profile = "S3DataPartnership438"
path = "s3://wbgggscecovid19dev-mobility/veraset/country=MM/year=*/date=*/*.parquet"
columns = ["uid", "datetime"]
ddf = dd.read_parquet(path, storage_options={"profile": profile}, columns=columns)

In [3]:
def to_local_time(df, hours, minutes, datetime_col):
    df["local_time"] = pd.to_datetime(df[datetime_col]) + timedelta(
        hours=hours, minutes=minutes
    )
    df["time_of_day"] = df.local_time.dt.time
    df["date"] = df.local_time.dt.date
    return df

In [4]:
ddf = ddf.map_partitions(to_local_time, 6, 30, "datetime")

In [5]:
ddf_no_dup = ddf.drop_duplicates(["uid", "date"])

In [ ]:
agg_by_day = ddf_no_dup.groupby("date").size().reset_index().compute()

In [ ]:
agg_by_day["date"] = pa.compute.cast(agg_by_day["date"].tolist(), pa.timestamp("s"))

In [ ]:
fig = px.line(
    agg_by_day.reset_index(),
    x="date",
    y="Users",
    title="Veraset users by day in Myanmar",
)
fig.update_yaxes(title_text="Number of users")
fig.show()